In [4]:
import tensorflow as tf
from tensorflow import keras
from hyperopt import fmin, tpe, hp, pyll
from hyperopt.pyll import scope
import hyperopt
from TweetDataProcessing.TweetDataReport import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from simpletransformers.language_representation import RepresentationModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.initializers import GlorotUniform, HeUniform
from tensorflow.keras.optimizers import Adam
np.set_printoptions(precision=8, suppress=False)
import pickle
import metrics
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output
import os
import re

In [12]:
with open('WF.pkl', 'rb') as f:  
    WF = pickle.load(f)

In [13]:
WF

,id,text,relevance,reps
0,897467574333710336,2007 . Κυβέρνηση Καραμανλή. 63 Άνθρωποι κάηκαν...,0,"[[-0.6408848, -0.48102167, 0.20622978, -0.0700..."
1,881453930370146304,56 δασικές πυρκαγιές μέσα σε ένα 24ωρο - σε εξ...,0,"[[-0.18210799, -0.5399046, 0.111069806, 0.3130..."
2,897201816047411200,#vouli #Καλαμο #Kalamos #Βαρναβα #φωτιά #πυρκα...,0,"[[-0.84237725, -0.2731679, -0.31454232, 0.5368..."
3,897871421879275520,#pyrkagia:Ευθύνες στους 300 διαχρονικά #vouli ...,0,"[[-0.5167864, -0.85462856, -0.17453858, 0.8997..."
4,897670763649662976,Αμυδρές ελπίδες ότι “ηρεμούν” οι φωτιές: Δεν έ...,1,"[[-0.6144835, -0.7360876, 0.08487149, 0.340718..."
...,...,...,...,...
3799,907235177885261824,Σε εξέλιξη μεγάλη πυρκαγιά στο Αγρίνιο | kontr...,1,"[[-0.44032797, -0.65290415, -0.13523479, 0.345..."
3800,897215964785332224,@couchpotatoes05 @pyrosvestiki Τραβάνε μεγάλο ...,0,"[[-0.7781397, -0.39640337, -0.5486824, 0.28955..."
3801,897214452738781184,#vouli #Καλαμο #Kalamos #Βαρναβα #φωτιά #πυρκα...,0,"[[-1.2320484, 0.09964307, -0.58601195, 0.54208..."
3802,907288819061231616,Σε εξέλιξη πυρκαγιά σε περιοχή του Ρεθύμνου h...,1,"[[-0.6051739, -0.53369755, 0.27293795, 0.63927..."


In [4]:
hyperspace = {
    'dropout_rate': hp.uniform('dropout_rate', 0.05, 0.45),
    'dropout_rate2': hp.uniform('dropout_rate2', 0.05, 0.45),
    'relovir': hp.uniform('relovir', 0.5, 1),
    'PostEmbedding#nods': hp.qnormal('PostEmbedding#nods',768,150,q = 1),
    'batch_size' : hp.quniform('batch_size',15,60,q=1),
    'PostEmbeddingActivation':hp.choice('PostEmbeddingActivation',['relu','tanh']),
    'PostEmbedding#nods2': hp.qnormal('PostEmbedding#nod2',768,150,q = 1),
    'PostEmbeddingActivation2':hp.choice('PostEmbeddingActivation2',['relu','tanh']),
    'PrePredictionActivation':hp.choice('PrePredictionActivation',['softmax','sigmoid'])
     }

In [7]:
def modelize(calibrers):
    
    # define manual/constant hyperparameters
    
    hyperparameters = {
                   'ITERATIONS':'20',
                   'EPOCHS': '50',
                   'LOSS':"'binary_crossentropy'",
                   'OPTIMIZER': 'Adam(learning_rate=0.0001)',
                   'METRICS': """['acc', metrics.precision, metrics.recall, metrics.f1]""",
                   'BATCH_SIZE': '32',
                   'TEST_SIZE': '0.15'}
    
#                    'RELOVIR': '1' }

    
    for key, value in hyperparameters.items():
        globals()[key] = eval(value)
        #print(key+':'+value)
    
    av_f1_val = np.zeros(EPOCHS)
    
    for iteration in range(ITERATIONS):
        
        # training set
        
        training_set_X, test_set_X, training_set_y, test_set_y = datasplit(WF, TEST_SIZE, relovir= calibrers['relovir'])
        
        #model building
        
        model = Sequential()
        model.add(Input(shape = (768,))) 
        model.add(Dense(calibrers['PostEmbedding#nods'],activation=calibrers['PostEmbeddingActivation']))
        model.add(Dropout(calibrers['dropout_rate']))
        model.add(Dense(calibrers['PostEmbedding#nods2'],activation=calibrers['PostEmbeddingActivation2']))
        model.add(Dropout(calibrers['dropout_rate2']))
        model.add(Dense(1,activation=calibrers['PrePredictionActivation']))
        
        
        
        model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)
        
        history = model.fit(training_set_X, training_set_y,validation_data=( test_set_X,  test_set_y), batch_size = int(calibrers['batch_size']), epochs = EPOCHS, verbose = 0,callbacks=[],shuffle = True)
    
        av_f1_val = np.add(av_f1_val,np.array(history.history['val_f1']))
 
    return -np.max(np.divide(av_f1_val,ITERATIONS))

In [8]:
best = fmin(fn=modelize, space=hyperspace, algo=tpe.suggest, max_evals=10)
best

100%|████████████████████████████████████████████| 10/10 [1:51:18<00:00, 667.86s/trial, best loss: -0.8732911765575408]


{'PostEmbedding#nod2': 816.0,
 'PostEmbedding#nods': 785.0,
 'PostEmbeddingActivation': 0,
 'PostEmbeddingActivation2': 1,
 'PrePredictionActivation': 1,
 'batch_size': 18.0,
 'dropout_rate': 0.27599333184153324,
 'dropout_rate2': 0.16804208635517714,
 'relovir': 0.6352668325536401}

In [9]:
model.summary()

NameError: name 'model' is not defined

In [8]:
best = fmin(fn=modelize, space=hyperspace, algo=tpe.suggest, max_evals=20)
best

100%|████████████████████████████████████████████| 20/20 [1:54:38<00:00, 343.93s/trial, best loss: -0.8636006504297257]


{'PostEmbedding#nods': 674.0,
 'PostEmbeddingActivation': 1,
 'PrePredictionActivation': 1,
 'batch_size': 52.0,
 'dropout_rate': 0.15166799040702791,
 'relovir': 0.6305057817313761}

In [34]:
best = fmin(fn=modelize, space=hyperspace, algo=tpe.suggest, max_evals=20)
best

100%|███████████████████████████████████████████████| 20/20 [21:36<00:00, 64.85s/trial, best loss: -0.8783693552017212]


{'PostEmbedding#nods': 553.0,
 'batch_size': 36.0,
 'dropout_rate': 0.11385248098147271,
 'relovir': 0.6138015752716723}

In [ ]:
def hoptimize(omniparameters):
    modelize()

In [6]:
best = fmin(fn=modelize, space=hyperspace, algo=tpe.suggest, max_evals=20)
best

100%|███████████████████████████████████████████████| 20/20 [16:53<00:00, 50.69s/trial, best loss: -0.8628330171108246]


{'dropout_rate': 0.12565449473968116,
 'relovir': 0.6317721961062578,
 'test_size': 0.23398836697583153}

# Big Greek Dataset

In [7]:
with open('greek_text.pkl', 'rb') as f:  
    WF = pickle.load(f)

In [8]:
hyperspace = {
    'dropout_rate': hp.uniform('dropout_rate', 0.05, 0.45),
    'dropout_rate2': hp.uniform('dropout_rate2', 0.05, 0.45),
    'relovir': hp.uniform('relovir', 0.5, 1),
    'PostEmbedding#nods': hp.qnormal('PostEmbedding#nods',768,150,q = 1),
    'batch_size' : hp.quniform('batch_size',15,60,q=1),
    'PostEmbeddingActivation':hp.choice('PostEmbeddingActivation',['relu','tanh']),
    'PostEmbedding#nods2': hp.qnormal('PostEmbedding#nod2',768,150,q = 1),
    'PostEmbeddingActivation2':hp.choice('PostEmbeddingActivation2',['relu','tanh']),
    'PrePredictionActivation':hp.choice('PrePredictionActivation',['softmax','sigmoid'])
     }

In [9]:
def modelize(calibrers):
    
    # define manual/constant hyperparameters
    
    hyperparameters = {
                   'ITERATIONS':'5',
                   'EPOCHS': '20',
                   'LOSS':"'binary_crossentropy'",
                   'OPTIMIZER': 'Adam(learning_rate=0.0001)',
                   'METRICS': """['acc', metrics.precision, metrics.recall, metrics.f1]""",
                   'BATCH_SIZE': '32',
                   'TEST_SIZE': '0.15'}
    
#                    'RELOVIR': '1' }

    
    for key, value in hyperparameters.items():
        globals()[key] = eval(value)
        #print(key+':'+value)
    
    av_f1_val = np.zeros(EPOCHS)
    
    for iteration in range(ITERATIONS):
        
        # training set
        
        training_set_X, test_set_X, training_set_y, test_set_y = datasplit(WF, TEST_SIZE, relovir= calibrers['relovir'])
        
        #model building
        
        model = Sequential()
        model.add(Input(shape = (768,))) 
        model.add(Dense(calibrers['PostEmbedding#nods'],activation=calibrers['PostEmbeddingActivation']))
        model.add(Dropout(calibrers['dropout_rate']))
        model.add(Dense(calibrers['PostEmbedding#nods2'],activation=calibrers['PostEmbeddingActivation2']))
        model.add(Dropout(calibrers['dropout_rate2']))
        model.add(Dense(1,activation=calibrers['PrePredictionActivation']))
        
        
        
        model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)
        
        history = model.fit(training_set_X, training_set_y,validation_data=( test_set_X,  test_set_y), batch_size = int(calibrers['batch_size']), epochs = EPOCHS, verbose = 0,callbacks=[],shuffle = True)
    
        av_f1_val = np.add(av_f1_val,np.array(history.history['val_f1']))
 
    return -np.max(np.divide(av_f1_val,ITERATIONS))

In [10]:
best = fmin(fn=modelize, space=hyperspace, algo=tpe.suggest, max_evals=3)
best

100%|████████████████████████████████████████████████| 3/3 [09:49<00:00, 196.65s/trial, best loss: -0.8569135308265686]


{'PostEmbedding#nod2': 696.0,
 'PostEmbedding#nods': 882.0,
 'PostEmbeddingActivation': 0,
 'PostEmbeddingActivation2': 0,
 'PrePredictionActivation': 1,
 'batch_size': 49.0,
 'dropout_rate': 0.11860873943025974,
 'dropout_rate2': 0.25017479664608383,
 'relovir': 0.5152784088265083}